In [ ]:
import numpy as np
import xarray as xr
from pathlib import Path
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import seaborn as sns

strength_cmap = sns.cubehelix_palette(start=0.5, rot=-0.5, as_cmap=True)

from sdm_eurec4a.visulization import (
    set_custom_rcParams,
    label_from_attrs,
    adjust_lightness_array,
    plot_one_one,
    handler_map_alpha,
)
from sdm_eurec4a.reductions import mean_and_stderror_of_mean

default_colors = set_custom_rcParams()
default_dark_colors = adjust_lightness_array(default_colors, 0.75)

In [ ]:
data_dir = Path("/home/m/m301096/CLEO/data/output_v4.0/")

eulerian_data_path = lambda microphysics: data_dir / Path(
    f"{microphysics}/combined/eulerian_dataset_combined.nc"
)
conservation_data_path = lambda microphysics: data_dir / Path(
    f"{microphysics}/combined/conservation_dataset_combined.nc"
)

In [ ]:
microphysics = (
    "null_microphysics",
    "condensation",
    "collision_condensation",
    "coalbure_condensation_small",
    "coalbure_condensation_large",
)
labels = dict(
    zip(
        microphysics,
        [
            "Null",
            "Evap only",
            "Coll",
            "CoalBuRe few",
            "CoalBuRe many",
        ],
    )
)

colors = dict(
    zip(
        microphysics,
        [
            "grey",
            "purple",
            "blue",
            "red",
            "orange",
            "red",
            "orange",
        ],
    )
)
dark_colors = dict(zip(microphysics, adjust_lightness_array(colors.values(), 0.75)))
light_colors = dict(zip(microphysics, adjust_lightness_array(colors.values(), 1.25)))
markers = dict(
    zip(
        microphysics,
        [
            "$*$",
            "1",
            "x",
            "+",
            "2",
            "3",
        ],
    )
)

In [ ]:
time_slice = slice(1500, 3490)

l = []
for microphysic in microphysics:
    print(microphysic)
    ds_euler_org = xr.open_dataset(eulerian_data_path(microphysic), chunks={})
    ds_euler = ds_euler_org[
        [
            "massdelta_condensation",
            "gridbox_volume",
            "gridbox_coord3",
            "relative_humidity",
            "max_gridbox",
            "liquid_water_content",
            "mass_moment1",
        ]
    ]
    ds_euler = ds_euler.sel(time=time_slice).mean("time", keep_attrs=True)
    ds_euler["surface_area"] = ds_euler_org["surface_area"].mean()
    ds_euler["surface_area"].attrs["units"] = "m^2"
    ds_euler["surface_area"].attrs["long_name"] = "Surface area"

    # add the mean and std of radius in mass space
    ds_select = ds_euler_org.sel(time=time_slice)
    mass_mean = ds_select["mass_represented"].mean("time", keep_attrs=True).compute()
    mass_mean = mass_mean.sel(gridbox=ds_euler_org["max_gridbox"])
    probability = mass_mean / mass_mean.sum("radius_bins", keep_attrs=True)
    probability = probability.fillna(0)
    mean_radius = (ds_select["radius_bins"] * probability).sum("radius_bins", keep_attrs=True)
    std_dev_radius: xr.DataArray = np.sqrt(
        (probability * (ds_select["radius_bins"] - mean_radius) ** 2).sum("radius_bins", keep_attrs=True)
    )
    mean_radius.attrs.update(
        units="µm",
        long_name="Mean radius of MSD",
        description="Mean radius of the mass size distribution",
    )
    std_dev_radius.attrs.update(
        units="µm",
        long_name="Standard deviation radius of MSD",
        description="Standard deviation radius of the mass size distribution",
    )

    ds_euler["precipitation"] = (
        ds_euler_org["precipitation"]
        .sum("radius_bins", keep_attrs=True)
        .sel(time=time_slice)
        .mean("time", keep_attrs=True)
    )

    ds_euler["mass_radius_mean"] = mean_radius
    ds_euler["mass_radius_std"] = std_dev_radius

    # from kg per dT to g per h per m^2
    ds_conser = xr.open_dataset(conservation_data_path(microphysic))
    ds_conser = ds_conser.sel(time=time_slice).mean("time", keep_attrs=True)

    ds_single = xr.merge([ds_conser, ds_euler])
    ds_single = ds_single.expand_dims(microphysics=(microphysic,))
    l.append(ds_single)
ds = xr.concat(l, dim="microphysics")


ds["max_gridbox"] = ds["max_gridbox"].fillna(ds["gridbox"].max())

ds["liquid_water_content"] = 1e3 * ds["liquid_water_content"]
ds["liquid_water_content"].attrs["units"] = "g/m^3"
ds["liquid_water_content"].attrs["long_name"] = "Liquid water content"

ds["cloud_liquid_water_content"] = ds["liquid_water_content"].sel(gridbox=ds["max_gridbox"])
ds["cloud_liquid_water_content"].attrs["long_name"] = "Cloud liquid water content"

ds["source"].attrs["long_name"] = "Evaporation"
ds["inflow"].attrs["long_name"] = "Cloud Base Precipitation Flux"
ds["outflow"].attrs["long_name"] = "Surface Precipitation Flux"

# from kg/m^3/s

# # to mg/m^3/h
# ds['evaporation_rate']  = - ds['massdelta_condensation'] * 1e6 * 3600
# ds['evaporation_rate'].attrs['units'] = 'mg/m^3/h'
# ds['evaporation_rate'].attrs['long_name'] = 'Evaporation rate'

# to mm/m/h
rho_water = 1000  # kg / m^3
ds["evaporation_rate"] = -1e3 / rho_water * ds["massdelta_condensation"] * 3600
ds["evaporation_rate"].attrs["units"] = "mm/h/m"
ds["evaporation_rate"].attrs["long_name"] = "Evaporation rate"

for var in ["source", "inflow", "outflow", "reservoir_change"]:
    attrs = ds[var].attrs.copy()
    # from  kg per dT per domain area
    # dT = 2s

    # # to    g per h per m^2
    # ds[var] = ds[var] / 2 * 3600 / ds['surface_area'] * 1e6
    # ds[var].attrs.update(attrs)
    # ds[var].attrs['units'] = 'mg/m^2/h'

    # to    mm / h
    rho_water = 1000  # kg / m^3
    ds[var] = ds[var] / 2 * 3600 / ds["surface_area"]  # kg / m^2 / h
    ds[var] = 1e3 * ds[var] / rho_water  # mm / h
    ds[var].attrs.update(attrs)
    ds[var].attrs["units"] = "mm/h"


ds["evaporation_fraction"] = -100 * ds["source"] / ds["inflow"]
ds["evaporation_fraction"].attrs["units"] = "\\%"
ds["evaporation_fraction"].attrs["long_name"] = "Evaporation fraction"

sorted_cloud_id = ds["cloud_id"].sortby(ds["evaporation_rate"].max(["gridbox", "microphysics"]))
# ds = ds.sel(cloud_id = sorted_cloud_id[:-2])
# ds_diff = ds - ds.sel(microphysics = 'condensation')

null_microphysics
condensation
collision_condensation
coalbure_condensation_small
coalbure_condensation_large


### Precipitation

In [ ]:
fig = plt.figure()
gs = fig.add_gridspec(nrows=1, ncols=1)

ax = fig.add_subplot(gs[:, :])

for mp in microphysics:

    ax.scatter(
        -ds.sel(microphysics=mp)["outflow"],
        ds.sel(microphysics=mp)["precipitation"],
        color=colors[mp],
        alpha=0.8,
        marker=markers[mp],
        label=labels[mp],
    )


ax.plot(
    -ds["outflow"],
    ds["precipitation"],
    color="grey",
    linewidth=0.2,
    alpha=0.5,
    zorder=0,
)


ax.legend(loc="upper left")

ax.set_xlabel("Calculated using the Box model attempt\n" + label_from_attrs(ds["outflow"]))
ax.set_ylabel("Calculated using the Eulerian attempt\n" + label_from_attrs(ds["precipitation"]))
plot_one_one(ax, color="black", linestyle="--")

### Evaporation rate

In [ ]:
y_ticks = np.arange(0, 1300, 250)

fig = plt.figure()
gs = fig.add_gridspec(nrows=2, ncols=4)

ax = fig.add_subplot(gs[:, :2])
ax0 = fig.add_subplot(gs[0, 2])
ax1 = fig.add_subplot(gs[0, 3], sharey=ax0, sharex=ax0)
ax2 = fig.add_subplot(gs[1, 2], sharey=ax0, sharex=ax0)
ax3 = fig.add_subplot(gs[1, 3], sharey=ax0, sharex=ax0)

axs = dict(
    zip(
        microphysics,
        [ax0, ax1, ax2, ax3],
    )
)

for mp in microphysics:

    md = ds.sel(microphysics=mp)["evaporation_rate"]
    md_mean, md_std = mean_and_stderror_of_mean(
        data=md,
        dims="cloud_id",
    )

    z = ds.sel(microphysics=mp)["gridbox_coord3"]
    z_mean, z_std = mean_and_stderror_of_mean(
        data=z,
        dims="cloud_id",
    )

    axs[mp].plot(
        md.T,
        z.T,
        color=light_colors[mp],
        alpha=0.3,
    )
    axs[mp].plot(
        md_mean,
        z_mean,
        label=labels[mp],
        color=dark_colors[mp],
    )

    axs[mp].set_yticks(y_ticks)
    axs[mp].set_yticklabels([])

    ax.plot(
        md_mean,
        z_mean,
        label=labels[mp],
        color=dark_colors[mp],
    )
    ax.fill_betweenx(
        z_mean,
        md_mean - md_std,
        md_mean + md_std,
        alpha=0.1,
        color=colors[mp],
    )
ax.set_yticks(y_ticks, y_ticks)
ax.legend()
# ax0.set_xlim(0, 500)
# ax.set_xlim(0, 130)

fig.supxlabel(label_from_attrs(ds["evaporation_rate"]))
fig.supylabel("Height [m]")
fig.suptitle("Evaporation profiles")
fig.tight_layout()

In [ ]:
y_ticks = np.arange(0, 1300, 250)

fig = plt.figure()
gs = fig.add_gridspec(nrows=2, ncols=4)

ax = fig.add_subplot(gs[:, :2])
ax0 = fig.add_subplot(gs[0, 2])
ax1 = fig.add_subplot(gs[0, 3], sharey=ax0, sharex=ax0)
ax2 = fig.add_subplot(gs[1, 2], sharey=ax0, sharex=ax0)
ax3 = fig.add_subplot(gs[1, 3], sharey=ax0, sharex=ax0)

axs = dict(
    zip(
        microphysics,
        [ax0, ax1, ax2, ax3],
    )
)

for mp in microphysics:

    md = ds_diff.sel(microphysics=mp)["evaporation_rate"]
    md_mean, md_std = mean_and_stderror_of_mean(
        data=md,
        dims="cloud_id",
    )

    z = ds.sel(microphysics=mp)["gridbox_coord3"]
    z_mean, z_std = mean_and_stderror_of_mean(
        data=z,
        dims="cloud_id",
    )

    axs[mp].plot(
        md.T,
        z.T,
        color=light_colors[mp],
        alpha=0.3,
    )
    axs[mp].plot(
        md_mean,
        z_mean,
        label=labels[mp],
        color=dark_colors[mp],
    )

    axs[mp].set_yticks(y_ticks)
    axs[mp].set_yticklabels([])
    # axs[mp].set_xscale('symlog', linthresh = 1e-8)
    ax.plot(
        md_mean,
        z_mean,
        label=labels[mp],
        color=dark_colors[mp],
    )
    ax.fill_betweenx(
        z_mean,
        md_mean - md_std,
        md_mean + md_std,
        alpha=0.3,
        color=colors[mp],
    )

ax0.set_xlim(-30, 55)

ax.set_yticks(y_ticks, y_ticks)

ax.set_xlim(-15, 55)
ax.legend(loc="upper right")

fig.supxlabel(label_from_attrs(ds["evaporation_rate"]))
fig.supylabel("Height [m]")
fig.suptitle("Evpoartion profiles | Difference to Evaporation only setup ")
fig.tight_layout()

In [ ]:
fig, axss = plt.subplots(1, 1, figsize=(10, 5))

axss = np.array(axss)
axs_flat = axss.flatten()

color_data = ds["evaporation_rate"]
y = ds["relative_humidity"]
x = ds["liquid_water_content"]


cmap = strength_cmap
norm = mcolors.LogNorm(
    vmin=color_data.compute().quantile(0.1).values, vmax=color_data.compute().quantile(0.9).values
)


for i, mp in enumerate(["condensation"]):

    c = color_data.sel(microphysics=mp)

    ax = axs_flat[i]

    sc = ax.scatter(
        x=x.sel(microphysics=mp),
        y=y.sel(microphysics=mp),
        c=c,
        cmap=cmap,
        norm=norm,
        marker=".",
        # label = labels[mp],
    )
    # ax.set_yscale('log')
    ax.set_xscale("log")
    ax.set_title(labels[mp])

cax = fig.add_axes([0.92, 0.1, 0.015, 0.8])
fig.colorbar(mappable=sc, cax=cax, label=label_from_attrs(color_data))
fig.supxlabel(label_from_attrs(x))
fig.supylabel(label_from_attrs(y))

Text(0.02, 0.5, 'Relative humidity $\\left[  \\%  \\right]$')

Use the inflow and outflow

In [ ]:
fig = plt.figure()
gs = fig.add_gridspec(nrows=1, ncols=1)

ax = fig.add_subplot(gs[:, :])

for mp in ds["microphysics"].values:

    data = ds.sel(microphysics=mp)["evaporation_fraction"]
    m, sem = mean_and_stderror_of_mean(
        data=data,
        dims="cloud_id",
    )

    label = f'{labels[mp]}: {m.values:.2f} ± {sem.values:.2f} ${data.attrs["units"]}$'

    ax.axvline(
        m,
        color=colors[mp],
        linestyle="-",
    )

    ax.hist(
        data,
        color=colors[mp],
        alpha=0.1,
        bins=np.arange(0, 100, 5),
        label=label,
    )

    ax.hist(
        data,
        color=colors[mp],
        alpha=1,
        linewidth=1,
        bins=np.arange(0, 100, 5),
        histtype="step",
    )


ax.legend(loc="upper right", handler_map=handler_map_alpha())
# ax.set_yscale('log')
# ax.set_xscale('log')


ax.set_xlabel(label_from_attrs(ds["evaporation_fraction"]))
ax.set_ylabel("Occansions")

Text(0, 0.5, 'Occansions')

In [ ]:
fig = plt.figure()
gs = fig.add_gridspec(nrows=1, ncols=1)

ax = fig.add_subplot(gs[:, :])

for mp in ds["microphysics"].values:

    ax.scatter(
        ds.sel(microphysics=mp)["inflow"],
        ds.sel(microphysics=mp)["evaporation_fraction"],
        color=colors[mp],
        alpha=0.8,
        marker=markers[mp],
        label=labels[mp],
    )

ax.legend(loc="upper left")
# ax.set_yscale('log')
ax.set_xscale("log")


ax.set_ylabel(label_from_attrs(ds["evaporation_fraction"]))
ax.set_xlabel(label_from_attrs(ds["inflow"]))

Text(0.5, 0, 'Cloud Base Precipitation Flux $\\left[  mm/h  \\right]$')

In [ ]:
fig = plt.figure()
gs = fig.add_gridspec(nrows=1, ncols=1)

ax = fig.add_subplot(gs[:, :])

for mp in ds["microphysics"].values:

    ax.scatter(
        ds["cloud_liquid_water_content"].sel(microphysics=mp),
        ds["evaporation_fraction"].sel(microphysics=mp),
        color=colors[mp],
        alpha=0.8,
        marker=markers[mp],
        label=labels[mp],
    )

ax.legend(loc="upper left")
# ax.set_yscale('log')
ax.set_xscale("log")


ax.set_ylabel(label_from_attrs(ds["evaporation_fraction"]))
ax.set_xlabel(label_from_attrs(ds["cloud_liquid_water_content"]))

Text(0.5, 0, 'Cloud liquid water content $\\left[  g/m^3  \\right]$')

('condensation',
 'collision_condensation',
 'coalbure_condensation_small',
 'coalbure_condensation_large')

In [ ]:
fig = plt.figure()
gs = fig.add_gridspec(nrows=1, ncols=1)

ax = fig.add_subplot(gs[:, :])

for mp in microphysics[1:]:

    ax.scatter(
        ds.sel(microphysics=mp)["inflow"],
        -ds.sel(microphysics=mp)["source"],
        color=colors[mp],
        alpha=0.8,
        marker=markers[mp],
        label=labels[mp],
    )

ax.plot(
    ds.sel(microphysics=list(microphysics[1:]))["inflow"],
    -ds.sel(microphysics=list(microphysics[1:]))["source"],
    color="grey",
    linewidth=0.2,
    alpha=0.5,
    zorder=0,
)


ax.legend(loc="upper left")


ax.set_xscale("log")
ax.set_yscale("log")
# ax.set_yscale('symlog', linthresh = 1e-6, linscale = 0.2)
lims = np.array([1e-6, 2.5e1])
ax.set_ylim(lims.min(), lims.max())
# ax.set_xlim(1e-4, lims.max())

p_x_values = np.geomspace(lims.min(), lims.max(), 100)


for p in [1, 0.1, 0.01]:
    style = dict(color="grey", alpha=p ** (1 / 5))
    lines = ax.plot(p_x_values, p * p_x_values, "--", linewidth=1, zorder=0, **style)
    line = lines[0]
    x = 15
    y = p * x
    # y = 1e-0
    # x =(1/1.3) * (y / p)
    ax.annotate(
        f"{100 * p:.0f} %",
        xy=(x, y),
        xytext=(0, 0),
        textcoords="offset points",
        va="top",
        ha="left",
        size=10,
        **style,
    )
    y = 2e-4
    x = y / p
    ax.annotate(
        f"{100 * p:.0f} %",
        xy=(x, y),
        xytext=(0, 0),
        textcoords="offset points",
        va="top",
        ha="left",
        size=10,
        **style,
    )

ax.set_xlabel(label_from_attrs(ds["inflow"]))
ax.set_ylabel(label_from_attrs(ds["source"]))

Text(0, 0.5, 'Evaporation $\\left[  mm/h  \\right]$')

In [ ]:
fig = plt.figure()
gs = fig.add_gridspec(nrows=1, ncols=1)

ax = fig.add_subplot(gs[:, :])

for mp in microphysics[1:]:

    ax.scatter(
        ds.sel(microphysics=mp)["inflow"],
        -ds.sel(microphysics=mp)["source"],
        color=colors[mp],
        alpha=0.8,
        marker=markers[mp],
        label=labels[mp],
    )

ax.plot(
    ds.sel(microphysics=list(microphysics[1:]))["inflow"],
    -ds.sel(microphysics=list(microphysics[1:]))["source"],
    color="grey",
    linewidth=0.2,
    alpha=0.5,
    zorder=0,
)


ax.legend(loc="upper left")


# ax.set_xscale('log')
# ax.set_yscale('log')
# ax.set_yscale('symlog', linthresh = 1e-6, linscale = 0.2)
lims = np.array([1e-6, 9])
ax.set_ylim(0, 1)
ax.set_xlim(0, 9)

p_x_values = np.geomspace(lims.min(), lims.max(), 100)


for p in [1, 0.1, 0.01]:
    style = dict(color="grey", alpha=p ** (1 / 5))
    lines = ax.plot(p_x_values, p * p_x_values, "--", linewidth=1, zorder=0, **style)
    line = lines[0]
    x = 7e0
    y = p * x
    # y = 1e-0
    # x =(1/1.3) * (y / p)
    ax.annotate(
        f"{100 * p:.0f} %",
        xy=(x, y),
        xytext=(0, 0),
        textcoords="offset points",
        va="center",
        ha="right",
        size=10,
        **style,
    )
    y = 1e-4
    x = 1.5 * (y / p)
    ax.annotate(
        f"{100 * p:.0f} %",
        xy=(x, y),
        xytext=(0, 0),
        textcoords="offset points",
        va="center",
        size=10,
        **style,
    )

ax.set_xlabel(label_from_attrs(ds["inflow"]))
ax.set_ylabel(label_from_attrs(ds["source"]))

Text(0, 0.5, 'Evaporation $\\left[  mm/h  \\right]$')

In [ ]:
fig = plt.figure()
gs = fig.add_gridspec(nrows=1, ncols=1)

ax = fig.add_subplot(gs[:, :])

for mp in ds["microphysics"].values:

    ax.scatter(
        ds.sel(microphysics=mp)["inflow"],
        -ds.sel(microphysics=mp)["outflow"],
        color=colors[mp],
        alpha=0.8,
        marker=markers[mp],
        label=labels[mp],
    )

ax.plot(
    ds["inflow"],
    -ds["outflow"],
    color="grey",
    linewidth=0.2,
    alpha=0.5,
    zorder=0,
)


ax.legend(loc="upper left")


ax.set_xscale("log")
ax.set_yscale("log")
# ax.set_yscale('symlog', linthresh = 1e-6, linscale = 0.2)
lims = np.array([1e-6, 2.5e1])
ax.set_ylim(lims.min(), lims.max())
ax.set_xlim(1e-4, lims.max())

p_x_values = np.geomspace(lims.min(), lims.max(), 100)


for p in [1, 0.1, 0.01]:
    style = dict(color="grey", alpha=p ** (1 / 5))
    lines = ax.plot(p_x_values, p * p_x_values, "--", linewidth=1, zorder=0, **style)
    line = lines[0]
    x = 7e0
    y = p * x
    # y = 1e-0
    # x =(1/1.3) * (y / p)
    ax.annotate(
        f"{100 * p:.0f} %",
        xy=(x, y),
        xytext=(0, 0),
        textcoords="offset points",
        va="center",
        ha="right",
        size=10,
        **style,
    )
    y = 1e-4
    x = 1.5 * (y / p)
    ax.annotate(
        f"{100 * p:.0f} %",
        xy=(x, y),
        xytext=(0, 0),
        textcoords="offset points",
        va="center",
        size=10,
        **style,
    )

ax.set_xlabel(label_from_attrs(ds["inflow"]))
ax.set_ylabel(label_from_attrs(ds["outflow"]))

Text(0, 0.5, 'Surface Precipitation Flux $\\left[  mm/h  \\right]$')

In [ ]:
fig = plt.figure()
gs = fig.add_gridspec(nrows=1, ncols=1)

ax = fig.add_subplot(gs[:, :])

A = ds["evaporation_fraction"]
B = ds.sel(microphysics="condensation")["evaporation_fraction"]

p = A - B

for mp in ds["microphysics"][1:].values:
    ax.scatter(
        ds.sel(microphysics=mp)["inflow"],
        p.sel(microphysics=mp),
        color=colors[mp],
        alpha=0.5,
        marker=markers[mp],
        label=labels[mp],
    )

    y, yerr = mean_and_stderror_of_mean(
        data=p.sel(microphysics=mp),
        dims="cloud_id",
    )
    x, xerr = mean_and_stderror_of_mean(
        data=ds.sel(microphysics=mp)["inflow"],
        dims="cloud_id",
    )

    ax.errorbar(
        x=x,
        y=y,
        xerr=xerr,
        yerr=yerr,
        color=dark_colors[mp],
    )

ax.plot(
    ds["inflow"],
    p,
    color="grey",
    linewidth=0.2,
    alpha=0.2,
    zorder=0,
)

ax.legend(loc="upper left")
ax.set_yscale("symlog", linthresh=1e-2, linscale=0.3)
ax.set_xscale("log")

yticks = np.array([100, 10, 1, 0.1, 0, -0.1, -1, -10, -100])

ax.set_yticks(yticks, yticks)

for p in yticks:
    style = dict(color="grey", alpha=np.abs(p / 100) ** (1 / 5))
    lines = ax.axhline(p, linestyle="--", linewidth=1, zorder=0, **style)

ax.axhline(0, color="grey", linewidth=0.5, zorder=0)

ax.set_title("Evaporated Fraction | Difference to Evaporation only")
ax.set_ylabel("Evaporated Fraction\nDifference to Evaporation only [%]")
ax.set_xlabel(label_from_attrs(ds["inflow"]))
# ax.set_yscale('symlog', linthresh = 1e-5)
# ax.set_ylim(0.90, 1.05)

Text(0.5, 0, 'Cloud Base Precipitation Flux $\\left[  mm/h  \\right]$')

In [ ]:
fig = plt.figure()

fig, axs = plt.subplots(1, 2, figsize=(10, 5))

for mp in microphysics:
    for _ax in axs:
        _ax.scatter(
            ds.sel(microphysics=mp)["inflow"],
            ds.sel(microphysics=mp)["cloud_liquid_water_content"],
            color=colors[mp],
            alpha=0.5,
            marker=markers[mp],
            label=labels[mp],
        )
for _ax in axs:
    _ax.plot(
        ds["inflow"],
        ds["cloud_liquid_water_content"],
        color="grey",
        linewidth=0.2,
        alpha=0.2,
        zorder=0,
    )

    _ax.legend(loc="upper left")
axs[1].set_yscale("log")
axs[1].set_xscale("log")

fig.supxlabel(label_from_attrs(ds["inflow"]))
fig.supylabel(label_from_attrs(ds["cloud_liquid_water_content"]))
fig.suptitle("Relation of Inflow and Cloud Liquid Water Content")
fig.tight_layout()

<Figure size 800x450 with 0 Axes>

In [ ]:
fig = plt.figure()
gs = fig.add_gridspec(nrows=1, ncols=1)

ax = fig.add_subplot(gs[:, :])

for mp in ds["microphysics"].values:

    ax.scatter(
        ds["mass_radius_mean"].sel(microphysics=mp),
        ds["evaporation_fraction"].sel(microphysics=mp),
        color=colors[mp],
        alpha=0.8,
        marker=markers[mp],
        label=labels[mp],
    )

ax.legend(loc="upper left")
# ax.set_yscale('log')
ax.set_xscale("log")

ax.set_xlabel(label_from_attrs(ds["mass_radius_mean"]))
ax.set_ylabel(label_from_attrs(ds["evaporation_fraction"]))

Text(0, 0.5, 'Evaporation fraction $\\left[  \\%  \\right]$')

In [ ]:
fig = plt.figure()
gs = fig.add_gridspec(nrows=1, ncols=1)

ax = fig.add_subplot(gs[:, :])

for mp in ds["microphysics"].values:

    ax.scatter(
        ds["cloud_liquid_water_content"].sel(microphysics=mp),
        -ds["source"].sel(microphysics=mp),
        color=colors[mp],
        alpha=0.8,
        marker=markers[mp],
        label=labels[mp],
    )

ax.legend(loc="upper left")
# ax.set_yscale('log')
ax.set_xscale("log")

ax.set_xlabel(label_from_attrs(ds["cloud_liquid_water_content"]))
ax.set_ylabel(label_from_attrs(ds["source"]))

Text(0, 0.5, 'Evaporation $\\left[  mm/h  \\right]$')